# Notebook 04 - Garages & Years for Build Dates  - Cleaning and fixing all Features

## Objectives
* Clean data
* Evaluate and process missing data
* Fix potential issues with data in given features:
    * GarageArea - Size of garage in square feet
    * GarageFinish - Interior Finish of the garage
    * GarageYrBlt - Year garage was built

## Inputs
* inputs/datasets/cleaning/basement.csv

## Outputs
* Clean and fix (missing and potentially wrong) data in given column
* After cleaning is completed, we will save current dataset in inputs/datasets/cleaning/garages_and_build_years.csv

## Change working directory
In This section we will get location of current directory and move one step up, to parent folder, so App will be accessing project folder.

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os

current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks/data_cleaning'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("you have set a new current directory")

you have set a new current directory


Confirm new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We need to check current working directory

In [4]:
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5/jupyter_notebooks'

We can see that current is **jupyter_notebooks**, as current notebook is in subfolder. We will go one step up to parent directory, what will be our project main directory.
Print out to confirm working directory

In [5]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

'/Users/pecukevicius/DataspellProjects/heritage_houses_p5'

## Loading Dataset

In [6]:
import pandas as pd

df = pd.read_parquet("inputs/datasets/cleaning/basement.parquet.gzip")
df.head()

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,0,856,854,3,No,706,GLQ,150,0.0,548,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1,1262,0,3,Gd,978,ALQ,284,NaN,460,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,2,920,866,3,Mn,486,GLQ,434,0.0,608,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,3,961,0,2,No,216,ALQ,540,NaN,642,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,4,1145,0,4,Av,655,GLQ,490,0.0,836,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


## Exploring Data

We will get all features that are missing data as a list, first we get given features datatypes

In [7]:
columns_of_interest = ['GarageArea', 'GarageFinish', 'GarageYrBlt']
column_types = df[columns_of_interest].dtypes

# Display the data types of these columns
df[columns_of_interest].dtypes

GarageArea        int64
GarageFinish     object
GarageYrBlt     float64
dtype: object

We will fix missing values:
1. GarageArea and GarageYrBlt missing values will be replaced with 0
2. GarageFinish missing values will be replaced with None

Also, we will convert GarageYrBlt from float to int

In [8]:
# Fill missing values for 'GarageArea' and 'GarageYrBlt' with 0
df[['GarageArea', 'GarageYrBlt']] = df[['GarageArea', 'GarageYrBlt']].fillna(0)

# Fill missing values for 'GarageFinish' with the string 'None'
df['GarageFinish'] = df['GarageFinish'].fillna('None')

# Convert 'GarageYrBlt' to integer
df['GarageYrBlt'] = df['GarageYrBlt'].astype(int)

df[columns_of_interest].head()

,GarageArea,GarageFinish,GarageYrBlt
0,548,RFn,2003
1,460,RFn,1976
2,608,RFn,2001
3,642,Unf,1998
4,836,RFn,2000


Now we will check, if there is no garage, all values should be 0 or None accordingly

We will reuse function from previous cleaning notebook - 03_basement.ipynb
Also we need to change features values to be inspected

In [9]:
def check_consistency(df, primary_feature):
    """
    Checks consistency of a primary feature against a set of expected values for related features.
    
    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        primary_feature (str): The primary feature to be checked.
    
    Returns:
        None: Outputs inconsistency results directly.
    """
    # Directly define features and their values indicating 'no presence' in a dictionary
    features_and_values = {
        "GarageArea": 0,
        "GarageFinish": 'None',
        "GarageYrBlt": 0
    }

    # Ensure primary feature is valid
    if primary_feature not in features_and_values:
        print(f"Feature {primary_feature} not defined in feature settings.")
        return

    # Determine the primary value to check against
    primary_value = features_and_values[primary_feature]

    # Check each feature against the primary feature's condition
    df['Consistency'] = df.apply(
        lambda row: True if row[primary_feature] != primary_value else all(
            row[feature] == value for feature, value in features_and_values.items() if feature != primary_feature
        ), axis=1
    )

    # Filter and display inconsistent records
    inconsistent_records = df[df['Consistency'] == False]
    return inconsistent_records

We loop through each feature and print the results

Again we will reuse code from previous notebook

In [10]:
def loop_check_consistency(df, basement_features):
    for feature in basement_features:
        errors = check_consistency(df, feature)
        error_count = errors.shape[0]  # Get the number of rows in the errors DataFrame
        print(f"Feature {feature} has {error_count} inconsistent rows.")


# Run the loop check consistency function
loop_check_consistency(df, columns_of_interest)

Feature GarageArea has 0 inconsistent rows.
Feature GarageFinish has 154 inconsistent rows.
Feature GarageYrBlt has 0 inconsistent rows.


We can see that all features are consistent except GarageFinish

Let's check that feature separately

In [11]:
garage_finish_check = check_consistency(df, 'GarageFinish')
garage_finish_check[columns_of_interest]

,GarageArea,GarageFinish,GarageYrBlt
7,484,None,1973
12,352,None,1962
23,572,None,1976
27,772,None,2008
35,691,None,2004
...,...,...,...
1401,398,None,2004
1415,660,None,2007
1438,923,None,1957
1442,812,None,2008


We can see that there is a lot of None, when we see that garage exist. Let's check is there any correlation between Garage Finish and other columns.
To achieve that, we will make a copy of dataframe, encode all objects as integers, then we will check correlations
Before that we need to encode it to numbers, to get correlations

In [12]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Create a copy of the DataFrame
df_encoded = df.copy()

# Initialize a LabelEncoder
encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
for column in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[column] = encoder.fit_transform(df_encoded[column])

# Check the transformed DataFrame
print(df_encoded.head())


   Unnamed: 0  1stFlrSF  2ndFlrSF  BedroomAbvGr  BsmtExposure  BsmtFinSF1  \
0           0       856       854             3             3         706   
1           1      1262         0             3             1         978   
2           2       920       866             3             2         486   
3           3       961         0             2             3         216   
4           4      1145         0             4             0         655   

   BsmtFinType1  BsmtUnfSF  EnclosedPorch  GarageArea  ...  MasVnrArea  \
0             2        150            0.0         548  ...       196.0   
1             0        284            NaN         460  ...         0.0   
2             2        434            0.0         608  ...       162.0   
3             0        540            NaN         642  ...         0.0   
4             2        490            0.0         836  ...       350.0   

   OpenPorchSF  OverallCond  OverallQual  TotalBsmtSF  WoodDeckSF  YearBuilt  \
0           

In [13]:
# Calculate Pearson correlation for 'GarageFinish' assuming it's still relevant
# If 'GarageFinish' is no longer a column, replace it with an appropriate column name
if 'GarageFinish' in df_encoded.columns:
    corr_pearson = df_encoded.corr(method='pearson')['GarageFinish'].sort_values(ascending=False, key=abs)[1:].head(10)
    print(corr_pearson)
else:
    print("'GarageFinish' is not in the DataFrame. Please replace it with a relevant column.")


YearBuilt       -0.422518
SalePrice       -0.403530
OverallQual     -0.382931
YearRemodAdd    -0.350866
KitchenQual      0.320148
GrLivArea       -0.277248
TotalBsmtSF     -0.266671
1stFlrSF        -0.263415
Consistency      0.232131
EnclosedPorch    0.205479
Name: GarageFinish, dtype: float64


In [14]:
# Calculate Pearson correlation for 'GarageFinish' assuming it's still relevant
# If 'GarageFinish' is no longer a column, replace it with an appropriate column name
if 'GarageFinish' in df_encoded.columns:
    corr_spearman = df_encoded.corr(method='spearman')['GarageFinish'].sort_values(ascending=False, key=abs)[1:].head(
        10)
    print(corr_spearman)
else:
    print("'GarageFinish' is not in the DataFrame. Please replace it with a relevant column.")


YearBuilt      -0.464056
SalePrice      -0.447545
OverallQual    -0.401935
YearRemodAdd   -0.370415
GarageYrBlt    -0.358334
KitchenQual     0.325532
GrLivArea      -0.291687
OpenPorchSF    -0.272318
TotalBsmtSF    -0.260371
Consistency     0.254695
Name: GarageFinish, dtype: float64


In [15]:
# Calculate Pearson correlation for 'GarageFinish' assuming it's still relevant
# If 'GarageFinish' is no longer a column, replace it with an appropriate column name
if 'GarageFinish' in df_encoded.columns:
    corr_kendall = df_encoded.corr(method='kendall')['GarageFinish'].sort_values(ascending=False, key=abs)[1:].head(10)
    print(corr_kendall)
else:
    print("'GarageFinish' is not in the DataFrame. Please replace it with a relevant column.")


YearBuilt      -0.353673
SalePrice      -0.341773
OverallQual    -0.335080
KitchenQual     0.291661
YearRemodAdd   -0.284825
GarageYrBlt    -0.271543
Consistency     0.233926
GrLivArea      -0.221186
OpenPorchSF    -0.218395
MasVnrArea     -0.202810
Name: GarageFinish, dtype: float64


We can not see any correlations between GarageFinish and other features. 

Now we will apply most common value in all dataset - GarageFinish

In [16]:
mode_value = df['GarageFinish'].mode()[0]  # mode() returns a Series; [0] accesses the first mode
mode_value

'Unf'

In [17]:
df.loc[garage_finish_check.index, 'GarageFinish'] = 'Unf'

Now lets check if there is anymore values that do not match

In [18]:
garage_finish_check = check_consistency(df, 'GarageFinish')
garage_finish_check[columns_of_interest]

,GarageArea,GarageFinish,GarageYrBlt


## GarageYrBlt fixing

We want to check when garages were build. Usually garages can not be built later than a house was build or remodeling:
1. We will filter buildings that have garage
2. We will filter buildings where garage build date is earlier that house build date

In [19]:
garage_year_mistake = df[
    (df['GarageYrBlt'] < df['YearBuilt']) & ((df['GarageFinish'] != 'None') | (df['GarageArea'] != 0))]
garage_year_mistake[['GarageYrBlt', 'YearBuilt', 'GarageFinish', 'GarageArea']]

,GarageYrBlt,YearBuilt,GarageFinish,GarageArea
29,1920,1927,Unf,240
93,1900,1910,Unf,506
324,1961,1967,Fin,564
600,2003,2005,Fin,736
736,1949,1950,Unf,400
1103,1954,1959,Fin,539
1376,1925,1930,Unf,160
1414,1922,1923,Unf,370
1418,1962,1963,Unf,336


We can see that there is 9 records, where garage was built (GarageYrblt) before House was built.
It is possible that garage was built on remodeling. Let's check Garage Year built lines up with house building and renovating dates (YearBuilt and YearRemodAdd accordingly)

In [20]:
garage_year_mistake[['GarageYrBlt', 'YearBuilt', 'GarageFinish', 'GarageArea', 'YearRemodAdd']]

,GarageYrBlt,YearBuilt,GarageFinish,GarageArea,YearRemodAdd
29,1920,1927,Unf,240,1950
93,1900,1910,Unf,506,1998
324,1961,1967,Fin,564,2007
600,2003,2005,Fin,736,2005
736,1949,1950,Unf,400,1950
1103,1954,1959,Fin,539,1959
1376,1925,1930,Unf,160,1950
1414,1922,1923,Unf,370,2000
1418,1962,1963,Unf,336,1963


Let's check, when most of the garages were built: on date of building or renovation

In [21]:
garages_built_same_as_house_year = df[(df['GarageYrBlt'] == df['YearBuilt']) & (df['GarageArea'] > 0)]
garages_built_same_as_renovation_year = df[(df['GarageYrBlt'] == df['YearRemodAdd']) & (df['GarageArea'] > 0)]

# Printing output
print("Garages built same as building: ", garages_built_same_as_house_year.shape[0])
print("Garages added during renovation: ", garages_built_same_as_renovation_year.shape[0])

Garages built same as building:  1089
Garages added during renovation:  725


We can see that most of them were built same time as building. but if we look more closely, we can see, that we have:
1088 + 725 = 1813, what is more than all records in dataset. It is possible, that renovation happened same year as house was built.
Let's check how many houses were renovated same yar as built

In [22]:
df[(df['GarageYrBlt'] == df['YearRemodAdd'])].shape[0]

725

This is very interesting, as we have 725 houses, which were renovated same date as they were built.
Let's check was there any renovations before houses were built

In [23]:
df[(df['GarageYrBlt'] > df['YearRemodAdd'])].shape[0]

127

We can see that there is 127 houses, which were renovated before they were built.

We need to check is there any NaN or zero values in House were build and renovated

In [24]:
print(df['YearBuilt'].isna().sum())  # Counts how many NaN values are in the 'YearBuilt' column
print(df['YearBuilt'].isnull().sum())  # Equivalent to isna(), also counts NaN values

print(df['YearRemodAdd'].isna().sum())  # Counts how many NaN values are in the 'YearRemodAdd' column
print(df['YearRemodAdd'].isnull().sum())  # Equivalent to isna(), also counts NaN values


0
0
0
0


We can see that there is no missing values or equal to 0, and we know that:
* All buildings were built between 1872 and 2010
* Same time we have information that renovation dates are also within same limits

We have to do next steps:
1. If Renovation date was between 1872 and 2010, but is smaller than build date, there was an error and data was entered in wrong cells, they need swapping
2. Filter out all buildings that have garage, and build date is NOT the same as renovation, then:
* we count how many garages were build same day as building
* we count how many garages were build during renovation
* Based on that, which ever is more - that is more common, and we will apply that date to garage build date for garages with wrong dates

In [25]:
df_tmp = df[(df['GarageYrBlt'] != df['YearRemodAdd'])]
df_tmp.shape[0]

735

In [26]:
garages_built_same_as_house_year = df[
    (df['GarageYrBlt'] == df['YearBuilt']) & (df['GarageArea'] > 0) & (df['GarageYrBlt'] != df['YearRemodAdd'])]
garages_built_same_as_renovation_year = df[
    (df['GarageYrBlt'] == df['YearRemodAdd']) & (df['GarageArea'] > 0) & (df['GarageYrBlt'] != df['YearRemodAdd'])]

# Printing output
print("Garages built same as building: ", garages_built_same_as_house_year.shape[0])
print("Garages added during renovation: ", garages_built_same_as_renovation_year.shape[0])

Garages built same as building:  418
Garages added during renovation:  0


We can see, that all Garages were build same time as building, none were added during Renovation.

Based on that, if garage year is lower than house build date, we will change garage build date to house build date

In [27]:
# Correcting garage build years that are earlier than the house build year
df.loc[df['GarageYrBlt'] < df['YearBuilt'], 'GarageYrBlt'] = df['YearBuilt']

Checking again is there any remaining Garage dates mistakes.

In [28]:
garage_year_mistake = df[
    (df['GarageYrBlt'] < df['YearBuilt']) & ((df['GarageFinish'] != 'None') | (df['GarageArea'] != 0))]
garage_year_mistake[['GarageYrBlt', 'YearBuilt', 'GarageFinish', 'GarageArea']]

,GarageYrBlt,YearBuilt,GarageFinish,GarageArea


All Information with Garages is cleaned and fixed, same time we have fixed house Build and Renovation dates.

## Removing added columns

We will use same code as in previous cleaning notebook 04_basement.ipynb

In [29]:
# Removing Extra columns that originally do not belong to dataset, as we have created them

df_original_features = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")
import pandas as pd

# Identify columns in df that are also in df_original
common_columns = df.columns.intersection(df_original_features.columns)

# Filter df to only include those common columns
df = df[common_columns]

df

,Unnamed: 0,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,0,856,854,3,No,706,GLQ,150,0.0,548,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1,1262,0,3,Gd,978,ALQ,284,NaN,460,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,2,920,866,3,Mn,486,GLQ,434,0.0,608,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,3,961,0,2,No,216,ALQ,540,NaN,642,...,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,4,1145,0,4,Av,655,GLQ,490,0.0,836,...,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,953,694,3,No,0,Unf,953,NaN,460,...,62.0,0.0,40,5,6,953,0.0,1999,2000,175000
1456,1456,2073,0,2,No,1028,ALQ,514,NaN,500,...,85.0,119.0,0,6,6,1542,NaN,1978,1988,210000
1457,1457,1188,1152,4,No,275,GLQ,877,NaN,252,...,66.0,0.0,60,9,7,1152,NaN,1941,2006,266500
1458,1458,1078,0,2,Mn,49,Unf,1029,112.0,240,...,68.0,0.0,0,6,5,1078,NaN,1950,1996,142125


## Saving current dataset

We will save current dataset as inputs/datasets/cleaning/garages_and_build_years.parquet.gzip

In [30]:
df.to_parquet('inputs/datasets/cleaning/garages_and_build_years.parquet.gzip', compression='gzip')

### Adding Cleaning code to pipeline
```python
# Direct assignments to fill missing values and convert data types for garage-related columns
df['GarageArea'] = df['GarageArea'].fillna(0)
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0).astype(int)
df['GarageFinish'] = df['GarageFinish'].fillna('None')

# Define a dictionary for checking consistency based on 'GarageFinish'
features_and_values = {"GarageArea": 0, "GarageFinish": 'None', "GarageYrBlt": 0}

def check_consistency(df, primary_feature):
    primary_value = features_and_values[primary_feature]
    return df.apply(
        lambda row: all(row[feature] == value for feature, value in features_and_values.items()) 
        if row[primary_feature] == primary_value else True, axis=1
    )

# Apply consistency check and correct 'GarageFinish'
consistency_mask = check_consistency(df, 'GarageFinish')
df.loc[~consistency_mask, 'GarageFinish'] = 'Unf'

# Correct garage years that are earlier than the house build year
df.loc[df['GarageYrBlt'] < df['YearBuilt'], 'GarageYrBlt'] = df['YearBuilt']
```

## Next step is cleaning Kitchen Quality - cleaning and fixing data in garages